In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from tqdm import tqdm_notebook

import sys
sys.path.append("..")
import utils

import rdkit
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
from rdkit.Chem.Draw import IPythonConsole

import matplotlib.pyplot as plt
%matplotlib inline

/Users/lmar3213/miniconda3/envs/lew_conda/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
interaction_matrix = sparse.load_npz('../data/interaction_matrix_ALL_species.npz')
targetNames = pd.read_csv('../data/all_targets', header=None).to_numpy().reshape(1,-1)[0]
targetIndex = (targetNames=='G-protein coupled receptor 55').nonzero()[0][0]

In [3]:
#test = sparse.lil_matrix((interaction_matrix.shape[0],interaction_matrix.shape[1]))
#test[:,targetIndex]=1
#test = sparse.csr_matrix(test)

In [ ]:
def calc_ranks_given_name(name, algo, train, test):
    if name == 'label_correlation':
        preds = algo(train)
    else:
        params = utils.read_params(name)
        preds = algo(params, train)
        for _ in range(7):
            preds += algo(params, train)
    ranks = utils.evaluate_predictions(preds, test, train)
    return ranks

filenames = ['label_correlation', 'hpo_implicit_bpr',
             'hpo_lightfm_warp']

##Functions to train those algorithms:
algorithms = [utils.train_label_correlation,
                  utils.train_implicit_bpr,
                  utils.train_lightfm_warp]


preds_list = list()
ranks_list = list()
for name, algo in zip(filenames, algorithms):
        print(name)
        if name == 'label_correlation':
            preds = algo(interaction_matrix)
        else:
            params = utils.read_params(name)
            preds = algo(params, interaction_matrix)
            for _ in range(7):
                preds += algo(params, interaction_matrix)
        preds_list.append(preds[:,targetIndex])

  0%|          | 238/200945 [00:00<01:24, 2374.35it/s]

label_correlation
y_in shape is: (630617, 852)


100%|██████████| 630617/630617 [01:31<00:00, 6909.48it/s]


hpo_implicit_bpr
hpo_lightfm_warp


In [ ]:
interaction_matrix

In [ ]:

tot_instances = np.array(interaction_matrix.sum(axis=0))[0]
row_mask = np.array(interaction_matrix.sum(axis=1)>=2).reshape(1,-1)[0]
    #y_in = y_in[row_mask]

In [ ]:
np.array(row_mask).reshape(1,-1)[0]

In [ ]:
np.array(interaction_matrix.sum(axis=1)<2).reshape(1,-1)[0]

In [ ]:
row_mask.sum()

In [ ]:
interaction_matrix[row_mask]